In [6]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

In [7]:
import pandas as pd
from datasets import Dataset, DatasetDict
import torchaudio

# Load Excel file
data = pd.read_excel('Aya_data.xlsx')
data['Transcription'] = data['Transcription'].str.strip()
data = data.dropna(subset=['Transcription'])
data = data.iloc[:, :3]
data['Transcription'] = data['Transcription'].str.replace('ث', 'س')
data = data.head(1850)

# Assuming your columns are 'Segment' and 'Transcription'
# data['path'] = 'Resampled_audio/' + data['Segment'] + '.wav'
data['audio'] = 'Resampled_audio/' + data['Segment'] + '.wav'

def load_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    return {
        'path': file_path,
        'array': waveform.squeeze().numpy(),
        'sampling_rate': sample_rate
    }

# Apply the load_audio function to each audio file in your dataset
data['audio'] = data['audio'].apply(load_audio)

# Rename columns to match the required format
data = data.rename(columns={'Transcription': 'sentence'})
data = data.drop(columns=['Segment','ID'])

# Convert to datasets format
dataset = Dataset.from_pandas(data)

# Split the dataset into train and test
dataset = dataset.train_test_split(test_size=0.1)

# Add the dataset to DatasetDict
dataset_dict = DatasetDict({
    'train': dataset['train'],
    'test': dataset['test']
})

# Verify the format
print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio', '__index_level_0__'],
        num_rows: 1662
    })
    test: Dataset({
        features: ['sentence', 'audio', '__index_level_0__'],
        num_rows: 185
    })
})


In [8]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


In [9]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="arabic", task="transcribe")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
input_str = dataset_dict["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 وأنا عمبتعرض لإشيا
Decoded w/ special:    <|startoftranscript|><|ar|><|transcribe|><|notimestamps|>وأنا عمبتعرض لإشيا<|endoftext|>
Decoded w/out special: وأنا عمبتعرض لإشيا
Are equal:             True


In [11]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="arabic", task="transcribe")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
dataset_dict["train"][0]

{'sentence': 'وأنا عمبتعرض لإشيا',
 'audio': {'array': [0.000213623046875,
   -0.00030517578125,
   0.00360107421875,
   0.00506591796875,
   0.002288818359375,
   0.0064697265625,
   -0.0009765625,
   -0.015045166015625,
   -0.02545166015625,
   -0.029052734375,
   -0.01690673828125,
   -0.004180908203125,
   0.006072998046875,
   0.010162353515625,
   0.0068359375,
   0.002197265625,
   -0.000762939453125,
   -0.00048828125,
   0.002532958984375,
   0.004913330078125,
   0.0086669921875,
   0.010101318359375,
   0.00628662109375,
   0.00347900390625,
   -0.00079345703125,
   -0.006103515625,
   -0.008758544921875,
   -0.0086669921875,
   -0.0048828125,
   0.006988525390625,
   0.00897216796875,
   0.004913330078125,
   -0.005340576171875,
   -0.014129638671875,
   -0.010955810546875,
   -0.016082763671875,
   -0.01898193359375,
   -0.024566650390625,
   -0.025726318359375,
   -0.0113525390625,
   0.001495361328125,
   0.01300048828125,
   0.01605224609375,
   0.012176513671875,
   0.

In [13]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [14]:
dataset_dict = dataset_dict.map(prepare_dataset,batch_size=4, remove_columns=dataset_dict.column_names["train"], num_proc=1)


Map:   0%|          | 0/1662 [00:00<?, ? examples/s]

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

In [15]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [16]:
model.generation_config.language = "arabic"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None


In [17]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [18]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

In [19]:

torch.cuda.is_available()


True

In [20]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)


In [21]:
import evaluate

metric = evaluate.load("wer")



Using the latest cached version of the module from C:\Users\PcUser\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--wer\85bee9e4216a78bb09b2d0d500f6af5c23da58f9210e661add540f5df6630fcd (last modified on Mon Jul 15 14:20:56 2024) since it couldn't be found locally at evaluate-metric--wer, or remotely on the Hugging Face Hub.


In [22]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [23]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./lebanese_dialect_asr",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=800,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=448,
    save_steps=10,
    eval_steps=10,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)


c:\Users\PcUser\anaconda3\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor, 
)


c:\Users\PcUser\anaconda3\Lib\site-packages\accelerate\accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [25]:
trainer.train()


  0%|          | 0/800 [00:00<?, ?it/s]

c:\Users\PcUser\anaconda3\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

  0%|          | 0/24 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 1.571264386177063, 'eval_wer': 89.32714617169374, 'eval_runtime': 113.3369, 'eval_samples_per_second': 1.632, 'eval_steps_per_second': 0.212, 'epoch': 0.1}


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 0.5180819034576416, 'eval_wer': 38.51508120649652, 'eval_runtime': 328.1901, 'eval_samples_per_second': 0.564, 'eval_steps_per_second': 0.073, 'epoch': 7.69}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


{'train_runtime': 38159.6446, 'train_samples_per_second': 0.335, 'train_steps_per_second': 0.021, 'train_loss': 0.20315536173991858, 'epoch': 7.69}


TrainOutput(global_step=800, training_loss=0.20315536173991858, metrics={'train_runtime': 38159.6446, 'train_samples_per_second': 0.335, 'train_steps_per_second': 0.021, 'total_flos': 3.68985292849152e+18, 'train_loss': 0.20315536173991858, 'epoch': 7.6923076923076925})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [26]:
kwargs = {
    # "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Whisper",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ar, split: test",
    "language": "ar",
    "model_name": "Whisper Small Arabic",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [27]:
trainer.push_to_hub(**kwargs)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

events.out.tfevents.1722697737.LAPTOP-1TFD0UPS.19920.14:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

events.out.tfevents.1722510878.LAPTOP-1TFD0UPS.19920.13:   0%|          | 0.00/11.7k [00:00<?, ?B/s]

Upload 18 LFS files:   0%|          | 0/18 [00:00<?, ?it/s]

...

events.out.tfevents.1722370116.LAPTOP-1TFD0UPS.19920.11:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

events.out.tfevents.1722459517.LAPTOP-1TFD0UPS.19920.12:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

events.out.tfevents.1730914600.LAPTOP-1TFD0UPS.26992.0:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AyaOsta/lebanese_dialect_asr/commit/0e4a668dc3b792b5a78d69c2ee7fd6b772d4026f', commit_message='End of training', commit_description='', oid='0e4a668dc3b792b5a78d69c2ee7fd6b772d4026f', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
tokenizer.push_to_hub("lebanese_dialect_asr")

README.md:   0%|          | 0.00/6.50k [00:00<?, ?B/s]

c:\Users\PcUser\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PcUser\.cache\huggingface\hub\models--AyaOsta--lebanese_dialect_asr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/AyaOsta/lebanese_dialect_asr/commit/0cecc336574311cd90a4c9b744bf7c378c4e723b', commit_message='Upload tokenizer', commit_description='', oid='0cecc336574311cd90a4c9b744bf7c378c4e723b', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="AyaOsta/lebanese_dialect_asr")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text




config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

c:\Users\PcUser\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PcUser\.cache\huggingface\hub\models--AyaOsta--lebanese_dialect_asr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.07k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/296k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/36.2k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(type="filepath"), 
    outputs="text",
    title="Lebanese Speech Recognition Model",
    description="Realtime demo for Arabic speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Users\PcUser\anaconda3\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.38.1, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
Traceback (most recent call last):
  File "c:\Users\PcUser\anaconda3\Lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\PcUser\anaconda3\Lib\site-packages\gradio\route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\PcUser\anaconda3\Lib\site-packages\gradio\blocks.py", line 1893, in process_api
    inputs = await self.preprocess_data(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\PcUser\anaconda3\Lib\site-packages\gradio\blocks.py", line 1625, in preprocess_data
    processed_input.append(block.preprocess(inputs_cached))
        